# Create Project Rephetio Neo4j Browser Guides

In [1]:
import os
import json
import subprocess
import re

import titlecase
import pandas

In [2]:
with open('template-full.txt') as read_file:
    template_full = read_file.read()
with open('template-mini.txt') as read_file:
    template_mini = read_file.read()

In [3]:
def format_df(df):
    for key in 'percent_of_prediction', 'percent_of_DWPC':
        if key in df.columns:
            df[key] = df[key].map('{:.2%}'.format)
    for key in 'path_count', 'distinct_metapaths':
        if key in df.columns:
            df[key] = df[key].map('{:,.0f}'.format)

    df.columns = df.columns.map(lambda x: titlecase.titlecase(x.replace('_', ' ')))
    return df

In [4]:
# Create a dictionary of (compound_id, disease_id) to clinical trial IDs
url = 'https://github.com/dhimmel/clintrials/raw/7c65dec7b69322ca2f8ba2b170c1b3dbd92ebff8/data/DrugBank-DO-slim.tsv'
clintrial_df = pandas.read_table(url)
cd_to_trials = {key: list(df.nct_id) for key, df in clintrial_df.groupby(['compound_id', 'disease_id'])}

In [5]:
predictions_df = pandas.read_table('../learn/prediction/predictions/probabilities.tsv')
pairs = list(zip(predictions_df.compound_id, predictions_df.disease_id))
len(pairs)

209168

In [6]:
# Load the dataframe of path counts
path_count_df = (
    pandas.read_table('../learn/prediction/features/dwpc.tsv.bz2')
    .query("hetnet == 'hetio-ind'")
    [['compound_id', 'disease_id', 'metapath', 'PC']]
    .rename(columns={'PC': 'path_count'})
)
pair_to_path_counts = dict(iter(path_count_df.groupby(['compound_id', 'disease_id'])))
path_count_df.head(2)

,compound_id,disease_id,metapath,path_count
0,DB01048,DOID:10652,CpDpCtD,0
1,DB01048,DOID:14330,CpDpCtD,0


In [7]:
# Load the dataframe of full metapath names
url = 'https://github.com/dhimmel/learn/raw/d2251a942813015d0362a90f179c961016336e77/prediction/features/metapaths.tsv'
metapath_name_df = (
    pandas.read_table(url)
    [['metapath', 'length', 'verbose']]
)
metapath_name_df.head(2)

,metapath,length,verbose
0,CbGaD,2,Compound–binds–Gene–associates–Disease
1,CbGdD,2,Compound–binds–Gene–downregulates–Disease


In [8]:
pattern = re.compile(r'([→—←–])')
def get_verbose_path(metapath_verbose, nodes_verbose):
    """
    Combine a verbose metapath and node string.
    
    Example:
    
    metapath_verbose: Octreotide—MPO—superior mesenteric artery—duodenum cancer
    nodes_verbose: Compound–binds–Gene–expresses–Anatomy–localizes–Disease
    returns: Octreotide–binds–MPO–expresses–superior mesenteric artery–localizes–duodenum cancer
    """
    metapath_split = re.split(pattern, metapath_verbose)
    path_split = re.split(pattern, nodes_verbose)
    for i, node in enumerate(path_split[::2]):
        metapath_split[i * 4] = node
    return ''.join(metapath_split)

In [9]:
def get_edge_df(df):
    s = pandas.Series()
    s['percent_of_prediction'] = df.percent_of_prediction.sum()
    s['path_count'] = len(df)
    s['distinct_metapaths'] = df.metapath.nunique()
    return s

In [10]:
def create_guide(compound_id, disease_id):
    params = {}
    directory = '../het.io-rep-data/prediction-info/{}/{}/'.format(compound_id, disease_id.replace(':', '_'))

    with open(os.path.join(directory, 'info.json')) as read_file:
        info = json.load(read_file)
    for key in 'compound_id', 'disease_id', 'compound_name', 'disease_name', 'prediction', 'compound_percentile', 'disease_percentile', 'category':
        if key in info:
            params[key] = info[key]
    params['fold_change'] = params['prediction'] / 0.00361 - 1
    for key in 'compound_percentile', 'disease_percentile':
        params[key] *= 100
    
    path_count_df = pair_to_path_counts[(compound_id, disease_id)][['metapath', 'path_count']]
    metapath_df = (
        pandas.Series(info['metapath_contribution'])
        .reset_index()
        .rename(columns = {'index': 'metapath', 0: 'percent_of_prediction'})
        .merge(path_count_df)
        .merge(metapath_name_df)
        .sort_values('percent_of_prediction', ascending=False)
        .pipe(format_df)
    )
    params['metapath_csv'] = metapath_df.to_csv(index=False).rstrip('\n')

    path = os.path.join(directory, 'highlights.cyp')
    if os.path.exists(path):
        with open(path) as read_file:
            params['path_query'] = read_file.read()
        full_path_df = pandas.read_table(os.path.join(directory, 'paths.tsv'))
        path_df = (full_path_df
            .head(25)
            .merge(metapath_name_df, how='left')
        )
        path_df['verbose_path'] = [get_verbose_path(*args) for args in zip(path_df.verbose, path_df.nodes)]
        path_df = path_df[['percent_of_prediction', 'percent_of_DWPC', 'metapath', 'length', 'verbose_path']].copy().pipe(format_df)
        params['path_csv'] = path_df.to_csv(index=False).rstrip('\n')
        
        # Edge tables
        for part in 'source', 'target':
            edge_df = (full_path_df
                .groupby(part + '_edge')
                .apply(get_edge_df).reset_index()
                .sort_values('percent_of_prediction', ascending=False)
                .head(25)
                .copy().pipe(format_df)
            )
            params[part + '_edge_csv'] = edge_df.to_csv(index=False).rstrip('\n')

    # Clinical Trials param
    trials = cd_to_trials.get((compound_id, disease_id), [])
    if trials:
        links = ', '.join('link:https://clinicaltrials.gov/ct2/show/{0}[[small]#{0}#]'.format(nct_id) for nct_id in trials)
        params['trial_sentence'] = 'ClinicalTrials.gov contains {n_trials} clinical trials investigating whether {compound_name} treats {disease_name} ({links}).'.format(links=links, n_trials=len(trials), **params)
    else:
        params['trial_sentence'] = 'No matching clinical trials were found in ClinicalTrials.gov.'

    # PharmacotherapyDB
    category = params.get('category')
    if category:
        indication_type = {'DM': 'disease-modifying ', 'SYM': 'symptomatic ', 'NOT': 'non-'}[category]
        params['phmcdb_sentence'] = 'In link:https://doi.org/10.15363/thinklab.d182[PharmacotherapyDB v1.0], {compound_name} is classified as a {indication_type}indication for {disease_name}.'.format(indication_type=indication_type, **params)
    else:
        params['phmcdb_sentence'] = 'link:https://doi.org/10.15363/thinklab.d182[PharmacotherapyDB v1.0] does not contain an indication between {compound_name} and {disease_name}.'.format(**params)
        
    
    #template = if 'path_query' in params else
    template = template_full if 'path_csv' in params else template_mini
    adoc = template.format(**params)
    with open('./guides/temp.adoc', 'wt') as write_file:
        write_file.write(adoc)

    path = os.path.join('..', 'guides', compound_id, '{}.html'.format(disease_id.replace(':', '_')))
    os.makedirs(os.path.dirname(path), exist_ok=True)
    subprocess.check_call("bash ./run.sh ../guides/temp.adoc {}".format(path),
        shell=True, cwd='neo4j-guides/')
    
    return params


In [11]:
%%time
for compound_id, disease_id in pairs:
    params = create_guide(compound_id, disease_id)

CPU times: user 7h 52min 26s, sys: 2h 51min 50s, total: 10h 44min 16s
Wall time: 23h 19min 48s


In [12]:
#cp guides/example.html /home/dhimmel/neo4j/hetionet-data/guides/example.html

In [13]:
#cp guides/DB01048/DOID_10652.html /home/dhimmel/neo4j/hetionet-data/guides/example.html

In [14]:
rm ./guides/temp.adoc

In [15]:
%%time
# Create a bzip2-tarball of guides
! tar --create --bzip2 --file guides.tar.bz2 guides

CPU times: user 40.3 s, sys: 6.1 s, total: 46.4 s
Wall time: 30min 43s
